## Logistic Regression 
We will use 1 vs all to train eighteen different logistic regression models with L2 regularization.  


In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time


#TODO 
#read in data 
trainX = np.loadtxt('spamtrainX.data')
trainY = np.loadtxt('spamtrainY.data')[:,np.newaxis]
testX = np.loadtxt('spamtestX.data')
testY = np.loadtxt('spamtestY.data')[:,np.newaxis]

### Loss, gradient of the loss, and Hessian of the loss for logistic regression.

\begin{align*}
L &= \left[\sum_{i=1}^m \ln(1+e^{-y_ix_i^\top w})\right] + \frac{\lambda}{2}\sum_{j=2}^n w_j^2
\end{align*}

\begin{align*}
p_i &= \sigma(y_ix_i^\top w) \\
\tilde{w} &= \begin{bmatrix} 0 & w_2 & w_3 & \dots & w_n \end{bmatrix}^\top \\
\tilde{I} &= \begin{bmatrix} 0 & 0 & 0 & & 0 \\
                        0 & 1 & 0 & \dots & 0 \\
                        0 & 0 & 1 & & 0 \\
                        & \vdots & & \ddots & \vdots \\
                        0 & 0 & 0 & \dots & 1 \end{bmatrix} \\
\nabla L &= - \left[\sum_{i=1}^m (1-p_i)y_i x_i^\top\right] + \lambda\tilde{w} \\
H_L &= \left[\sum_{i=1}^m p_i(1-p_i)x_ix_i^\top\right] + \lambda\tilde{I}
\end{align*}


In [ ]:
# Write your code to calculate the same here

def lrloss(w,X,Y,lmbda):
    wnob = w.copy()
    wnob[0] = 0
    return (np.sum(np.log(1+np.exp(-Y*(X@w))),0).T + 0.5*lmbda*np.sum(wnob*wnob))[0]

def lrgrad(w,X,Y,lmbda):
    wnob = w.copy()
    wnob[0] = 0
    return -np.sum((1-1/(1+np.exp(-Y*(X@w))))*X*Y,0)[:,np.newaxis] + lmbda*wnob

def lrhess(w,X,Y,lmbda):
    p = 1/(1+np.exp(-Y*(X@w)))
    ey = np.eye(w.shape[0])
    ey[0,0] = 0
    R = p*(1-p)
    return X.T@np.diag(R[:,0])@X + lmbda*ey

### Newton-Raphson minimization.  


In [ ]:
# gradient descent implemented with a constant step size
# (note: this is *not* a good implementation, but just to show the idea)
# The "ittfn" can help with debugging, but isn't necessary
def graddesc(w,eta,fn,gradfn,ittfn=None):
    oldf = fn(w)
    df = 1
    while(df>1e-6):
        w = w - eta*gradfn(w)
        newf = fn(w)
        df = oldf-newf # hope to be positive, or we've over-shot and will be done
        if ittfn is not None:
            ittfn(w,eta,newf)
        oldf = newf
    return w
    
# Newton-Raphson minimization
def newton(w,fn,gradfn,hessfn,ittfn=None):
    oldf = fn(w)
    eta = 1
    while True:
        g = gradfn(w)
        H = hessfn(w)
        neww = w - np.linalg.solve(H,g)
        newf = fn(neww)
        if newf>=oldf:
            usedg = True
            eta *= 2
            while eta>1e-10:
                neww = w - eta*g
                newf = fn(neww)
                if newf<oldf:
                    break
                eta *= 0.5
            if eta<1e-10:
                return w
        else:
            usedg = False
        oldf = newf
        w = neww
        if ittfn is not None:
            if usedg:
                ittfn(w,eta,oldf)
            else:
                ittfn(w,0,oldf)

### Train and check error rate 

In [ ]:
def trainlr(X,Y,lmbda):
    w0 = np.zeros((X.shape[1],1)) # starting w at zero works well for LR
    return newton(w0,lambda w : lrloss(w,X,Y,lmbda),
                  lambda w : lrgrad(w,X,Y,lmbda),
                  lambda w : lrhess(w,X,Y,lmbda))

def lrerrorrate(X,Y,w):
    return np.sum(Y*X@w<0)/Y.shape[0]

lmbda = 0.1
useextrafeatures = True

myw = trainlr(mytrainX,trainY,lmbda)
#myw = (myw>1.0).astype(float)
#print (myw.T)
print (lrerrorrate(mytestX,testY,myw))

### 3-fold cross validation
Do not need to do this 

In [ ]:
def xvalideval(X,Y,lmbda,extrafeatures):
    nfold = 3
    tX = phi(X,extrafeatures)
    def evalfn(X,Y,eX,eY):
        return lrerrorrate(eX,eY,trainlr(X,Y,lmbda))
        
    m = X.shape[0]
    splits = np.linspace(0,m,nfold+1).astype(int)
    v = 0
    for low,high in zip(splits[0:-1],splits[1:]):
        trainX = np.vstack((tX[:low,:],tX[high:,:]))
        trainY = np.vstack((Y[:low,:],Y[high:,:]))
        validX = tX[low:high,:]
        validY = Y[low:high,:]
        v +=evalfn(trainX,trainY,validX,validY)
    return v

ls = np.logspace(-4,2,10)
xverr = ls.copy()
bstv = None
bste = None
bstl = None
for i,l in enumerate(ls):
    xverr[i] = xvalideval(trainX,trainY,l,False)
    if bstv is None or bstv>xverr[i]:
        bstv = xverr[i]
        bstl = l
        bste = False
plt.semilogx(ls,xverr,'b-')
for i,l in enumerate(ls):
    xverr[i] = xvalideval(trainX,trainY,l,True)
    if bstv is None or bstv>xverr[i]:
        bstv = xverr[i]
        bstl = l
        bste = True
plt.semilogx(ls,xverr,'g-')
plt.legend(['raw features','raw + binary features'])
